# The Model

## setup

### input data

In [ ]:
## imports
#  modules
import numpy as np
import pandas as pd
import xarray as xr
import warnings
import os

# custom code
import dask_addons
from open_data import open_data_dir, data_dir_to_zarr
from neural_net import DNN, geoDataset
from open_fire_data import FlattenedTruthTable

warnings.filterwarnings('ignore')

In [12]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:63475/status,
Dashboard: http://127.0.0.1:63475/status,Workers: 4
Total threads: 8,Total memory: 15.52 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:63476,Workers: 0
Dashboard: http://127.0.0.1:63475/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:63504,Total threads: 2
Dashboard: http://127.0.0.1:63505/status,Memory: 3.88 GiB
Nanny: tcp://127.0.0.1:63479,


In [13]:
DATA_DIR_PATH = '../data/_ZARR_FILES'

data_path = os.path.join(DATA_DIR_PATH, 'alaska_full.zarr')
prior_data_path = os.path.join(DATA_DIR_PATH, 'alaska_prior.zarr')

data = xr.open_zarr(data_path, decode_timedelta=False)
prior_data = xr.open_zarr(prior_data_path, decode_timedelta=False)

In [14]:
input_ds = dask_addons.FlattenedDaskDataset(data, prior_data)
input_ds.setup()

[compute_running_totals] run time: 0.3961753845214844s


### fire data

In [16]:
fire_data = FlattenedTruthTable(pd.read_csv('../data/_FIRE/alaska_range_csv/data.csv'))

### dataset

In [18]:
ds = geoDataset(input_ds, fire_data, feature_num = 13)

## training


In [ ]:
import torch
from neural_net import train

model = DNN()
print(model)

DNN(
  (network): Sequential(
    (0): Linear(in_features=13, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): ReLU()
    (4): Linear(in_features=10, out_features=1, bias=True)
    (5): Sigmoid()
  )
)
